In [60]:
import os
from flask import Flask
import json
import time
import auth
import pytz
import socket
import sqlite3
import numpy as np
import datetime
import pandas as pd
from jinja2 import Environment, FileSystemLoader
from pandas.io.json import json_normalize
from flask_sqlalchemy import SQLAlchemy
pd.set_option('display.max_colwidth', -1)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

def get_datax(_db):
    conn = sqlite3.connect(_db)
    cur = conn.cursor()
    cur.execute("SELECT json_data FROM flight_data limit 1")
    _data = cur.fetchall()
    cur.close()
    conn.close()
    return _data

def determine_direction(x):
    """

    :param x:
    :return:
    """
    x = str(x)
    if x == "BTS" or x == "KSC" or x == "SLD" or x == "TAT":
        return str("TAM")
    else:
        return str("SPAT")


def determine_production(x):
    """

    :param x:
    :return:
    """
    x = str(x)
    if x == "BTS":
        return str("BTS")
    elif x == "KSC":
        return str("KSC")
    else:
        return str("---")


def extra_catering(x):
    """

    :param x:
    :return:
    """
    try:
        return ['{0}: {1}'.format(i['code'],i['count']) for i in x]
    except:
        return 0


def split_weird_timeframes(_i, _dataframe):
    """

    :param _i:
    :param _dataframe:
    :return:
    """
    _s = _i.split('___')[0]
    _e = _i.split('___')[1]
    return _dataframe.loc[_s:_e]


def add_one_day(_ii):
    _plus_day = datetime.datetime.strptime(_ii, '%Y-%m-%d %H:%M:%S') + datetime.timedelta(1)
    return _plus_day



def get_unique_days(_data):
    """

    :param _data:
    :return:
    """
    _raw_data = json.loads((_data[0][0]).decode("utf-8"))
    _df = json_normalize(_raw_data['data']['flight']['data'])
    
    # _df = json_normalize(_data['data']['flight']['data'])
    _df_normalized = _df
    
    _all_unique_days = _df['local_std_date'].unique()
    _all_unique_days = np.sort(_all_unique_days)
    _all_unique_days = ['{}___{}'.format(_i + ' 09:00:00', add_one_day(_i + ' 09:00:00')) for _i in _all_unique_days]
    _all_unique_days[0] = str(datetime.datetime.strptime(_all_unique_days[0].split('___')[0], '%Y-%m-%d %H:%M:%S') \
                          - datetime.timedelta(hours=9)) + "___" + str(_all_unique_days[0].split('___')[1])
    print(_all_unique_days)
    return _all_unique_days, _df_normalized

def render_tables(_df_normalized):
    """

    :param _df_normalized:
    :return:
    """
    _df = _df_normalized
    _compare = pd.DataFrame()

    _compare['Departure'] = pd.to_datetime(_df['local_std_date'] + ' ' + _df['local_std_time'])
    _compare['Depart'] = _compare['Departure']
    _compare['Arrival'] = pd.to_datetime(_df['local_sta_date'] + ' ' + _df['local_sta_time'])
    _compare['Flight'] = _df['flight_number']
    _compare['Aircraft'] = _df['aircraft_config']
    _compare['Reg'] = _df['aircraft_reg']
    _compare['Route'] = _df['route_id']
    _compare["Meal"] = _df["catering_order.flight_meal_type"]
    _compare['Direction'] = _df['departure_iata'].map(determine_direction)
    _compare['Production'] = _df['departure_iata'].map(determine_production)
    _compare['From'] = _df['departure_iata']
    _compare['To'] = _df['destination_iata']

    _compare['Quantity'] = _df['catering_order.quantity_y']
    _compare['Crew'] = _df['catering_order.quantity_crew']
    _compare['Extra Catering'] = _df['extra_catering'].map(extra_catering)
    _compare['Note'] = _df['catering_order.general_note'].map(
        lambda x: "{0}{1}{2}".format("""<div class="hoverable">""", str(x), "</div>"))
    _compare['Note'] = _compare['Note'].map(lambda x: str(x).replace('\n', "<br>"))
    _compare['Note'] = _compare['Note'].map(lambda x: str(x).replace('\r', ''))
    _compare['Quantity'] = _compare['Quantity'].fillna(0)
    _compare['Crew'] = _compare['Crew'].fillna(0)
    _compare.index = _compare['Departure']
    return _compare



In [61]:
db = "2w.sqlite"
data = get_datax(db)
ud, normal = get_unique_days(data)
compare = render_tables(normal)

['2018-07-17 00:00:00___2018-07-18 09:00:00', '2018-07-18 09:00:00___2018-07-19 09:00:00', '2018-07-19 09:00:00___2018-07-20 09:00:00', '2018-07-20 09:00:00___2018-07-21 09:00:00', '2018-07-21 09:00:00___2018-07-22 09:00:00']
